In [0]:
%sql
USE CATALOG etl_project;
USE SCHEMA bigquery_db_cohort_db;

-- Build the Silver order facts table with order-level metrics and recency features

CREATE OR REPLACE TABLE silver_order_facts AS

-- 1) Base enrichment: normalized time fields and order sequence per customer
WITH base AS (
    SELECT
        order_id,
        customer_id,
        order_date,

        -- Normalized time fields for monthly/yearly aggregation
        to_date(DATE_TRUNC('month', order_date)) AS order_month,
        YEAR(order_date) AS order_year,

        -- Sequential order number per customer (1 = first order)
        ROW_NUMBER() OVER (
            PARTITION BY customer_id
            ORDER BY order_date, order_id
        ) AS order_number_per_customer,

        -- Previous order date per customer (NULL for first order)
        LAG(order_date) OVER (
            PARTITION BY customer_id
            ORDER BY order_date, order_id
        ) AS previous_order_date
    FROM bronze_ecom_orders
),

-- 2) Enrich with first/repeat flags and recency in days
enriched AS (
    SELECT
        order_id,
        customer_id,
        order_date,
        order_month,
        order_year,
        order_number_per_customer,
        previous_order_date,

        -- Flag first-time customers (first order only)
        CASE WHEN order_number_per_customer = 1 THEN TRUE ELSE FALSE END AS is_first_order,
        -- Flag repeat purchases (2nd order and beyond)
        CASE WHEN order_number_per_customer > 1 THEN TRUE ELSE FALSE END AS is_repeat_order,

        -- Days since last order; NULL for first order
        CASE
            WHEN previous_order_date IS NULL THEN NULL
            ELSE date_diff(DAY, previous_order_date, order_date)
        END AS days_since_last_order
    FROM base
)

-- 3) Expose all enriched fields as Silver order facts
SELECT *
FROM enriched;

